In [2]:
import sys
from datetime import date
from pathlib import Path
import pickle
from typing import List, Tuple

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import Input, Model

sys.path.append("..")
from btc_predictor.config import config, logging_config
from btc_predictor.datasets import BitfinexCandlesAPIData
from btc_predictor.models import LSTMBTCPredictor

# from btc_predictor.datasets import DataReader
# from btc_predictor.models import LSTM_Model
# from btc_predictor.utils import print_metrics
# from btc_predictor.utils import show_plot
# from btc_predictor.utils import plot_train_history

tf.random.set_seed(78)
np.random.seed(78)
mpl.rcParams['figure.figsize'] = [16.0, 9.0]
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
candles = BitfinexCandlesAPIData()
candles.load(start_time=1610000000000)
df = candles.pd

In [7]:
BTCUSD = df.copy()

BTCUSD['baseline_close'] = BTCUSD['close'].shift(1)
BTCUSD['log_ret'] = np.log(BTCUSD['close']) - np.log(BTCUSD['close'].shift(1))
BTCUSD['baseline_log_ret'] = np.log(BTCUSD['baseline_close']) - np.log(BTCUSD['baseline_close'].shift(1))
BTCUSD.dropna(inplace=True)
print(f'Total daily data: {BTCUSD.shape[0]} days')

Total daily data: 9998 days


# 1. Univariate RNN Model

Our best statistical model is ARIMA(5, 1, 1) was trained using 330 days of closing log returns and forcasted on forward 7 days. To make models comparable, we will also try to forecast the forward 7 days log returns. We tried to use 0th order differencing but the model failed to learn. Our ARIMA model provided the hit of using 1st order differencing thus we will apply the same thing.

In [6]:
time_series_data = np.diff(BTCUSD['log_ret'].to_numpy()).astype('float32')
train = time_series_data[:config.TRAIN_SIZE]
val = time_series_data[config.TRAIN_SIZE:config.VAL_SIZE+config.TRAIN_SIZE]
test = time_series_data[config.VAL_SIZE+config.TRAIN_SIZE:]

## 1.1 Data Preprocessing using tf.data

In [7]:
train_tfds = data.create_tfds_from_np(data=train,
                                      window_size=config.WINDOW_SIZE,
                                      batch_size=config.BATCH_SIZE)
val_tfds = data.create_tfds_from_np(data=val,
                                    window_size=config.WINDOW_SIZE,
                                    batch_size=config.BATCH_SIZE)
test_tfds = data.create_tfds_from_np(data=test,
                                     window_size=config.WINDOW_SIZE,
                                     batch_size=1)